In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

def prepare_output(input, predictions, year, month):
    df = pd.DataFrame()
    df['ride_id'] = f'{year:04d}/{month:02d}_' + input.index.astype('str')
    df['predictions'] = predictions
    return df

In [5]:
year = 2021
month = 2

In [6]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [8]:
pred_avg = sum(y_pred) / len(y_pred)
print(f'Average prediction for {year:04d}-{month:02d} is {pred_avg:.2f}')

Average prediction for 2021-02 is 16.19


In [9]:
!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [10]:
df_result = prepare_output(input=df, predictions=y_pred, year=year, month=month)
df_result.to_parquet(
    "output/h4q2.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

In [11]:
!ls -la output

total 19260
drwxrwxr-x 2 ubuntu ubuntu     4096 Jun 25 18:30 .
drwxrwxr-x 3 ubuntu ubuntu     4096 Jun 25 18:33 ..
-rw-rw-r-- 1 ubuntu ubuntu 19711440 Jun 25 18:35 h4q2.parquet
